# Time-Dependent Subduction Zone Setup

Authors: Cameron Seebeck, Cian Wilson

## Implementation

Recalling our implementation strategy we are following a similar workflow to that seen in the [background examples](../02_background/2.1_background_intro.ipynb).

1. we will describe the subduction zone geometry and tesselate it into non-overlapping triangles to create a **mesh**
2. we will declare **function spaces** for the temperature, wedge velocity and pressure, and slab velocity and pressure
3. using these function space we will declare **trial** and **test functions**
4. we will define Dirichlet boundary conditions at the boundaries as described in the [introduction](./3.1_sz_intro.ipynb)
5. we will describe **discrete weak forms** for temperature and each of the coupled velocity-pressure systems that will be used to assemble the matrices (and vectors) to be solved
6. we will set up matrices and solvers for the discrete systems of equations
7. we will solve the matrix problems

For the time-dependent cases we have now implemented all but the rheology specific final step of solving the coupled velocity-pressure-temperature problem.  In this notebook we do this for the case of an isoviscous rheology, deriving a new `TDIsoSubductionProblem` class from the `TDSubductionProblem` class we implemented in [`notebooks/03_sz_problems/3.5a_sz_tdep_problem.ipynb`](./3.5a_sz_tdep_problem.ipynb).

## Preamble

Let's start by adding the path to the modules in the `python` folder to the system path (so we can find the our custom modules).

In [ ]:
import sys, os
basedir = ''
if "__file__" in globals(): basedir = os.path.dirname(__file__)
sys.path.append(os.path.join(basedir, os.path.pardir, os.path.pardir, 'python'))

Let's also load the module generated by the previous notebooks to get access to the parameters and functions defined there.

In [ ]:
from sz_problems.sz_params import default_params, allsz_params
from sz_problems.sz_slab import create_slab
from sz_problems.sz_geometry import create_sz_geometry
from sz_problems.sz_problem import TemperatureSolver
from sz_problems.sz_tdep_problem import TDSubductionProblem

Then let's load all the required modules at the beginning.

In [ ]:
import geometry as geo
import utils
from mpi4py import MPI
import dolfinx as df
import dolfinx.fem.petsc
from petsc4py import PETSc
import numpy as np
import scipy as sp
import ufl
import basix.ufl as bu
import matplotlib.pyplot as pl
import copy
import pyvista as pv
import pathlib
output_folder = pathlib.Path(os.path.join(basedir, "output"))
output_folder.mkdir(exist_ok=True, parents=True)

## `TDIsoSubductionProblem` class

We build on the `TDSubductionProblem` class implemented in [`notebooks/03_sz_problems/3.5a_sz_tdep_problem.ipynb`](./3.5a_sz_tdep_problem.ipynb), deriving a `TDIsoSubductionProblem` class that implements and solves the equations for a time-dependent isoviscous case.

### 7. Solution

In the isoviscous case, $\eta$ = 1 (`eta = 1`), only the temperature depends on time and the velocity (and not vice-versa).  So to solve the full time-dependent system of equations we only need to solve the two velocity-pressure systems once (already implemented in `SubductionProblem.solve_stokes_isoviscous`) before solving the temperature repeatedly, incrementing the time after each solve in a time-loop, to get a fully evolved solution for all variables (implemented below in `solve`).  The timestep is controlled with the parameter `dt` and should be chosen in conjunction with `theta` to ensure a stable solution is found.  The time-loop finishes once the final time, `tf`, is reached.

In [ ]:
class TDIsoSubductionProblem(TDSubductionProblem):
    def solve(self, tf, dt, theta=0.5, verbosity=2, 
              petsc_options_s=None, petsc_options_T=None, plotter=None):
        """
        Solve the coupled temperature-velocity-pressure problem assuming an isoviscous rheology with time dependency

        Arguments:
          * tf - final time  (in Myr)
          * dt - the timestep (in Myr)
          
        Keyword Arguments:
          * theta         - theta parameter for timestepping (0 <= theta <= 1, defaults to theta=0.5)
          * petsc_options - a dictionary of petsc options to pass to the solver (defaults to mumps)
          * verbosity     - level of verbosity (<1=silent, >0=basic, >1=timestep, defaults to 2)
          * petsc_options_s - a dictionary of petsc options to pass to the Stokes solver 
                              (defaults to an LU direct solver using the MUMPS library) 
          * petsc_options_T - a dictionary of petsc options to pass to the temperature solver 
                              (defaults to an LU direct solver using the MUMPS library)
        """
        assert(theta >= 0 and theta <= 1)
        
        # set the timestepping options based on the arguments
        # these need to be set before calling self.temperature_forms_timedependent
        self.dt = df.fem.Constant(self.mesh, df.default_scalar_type(dt/self.t0_Myr))
        self.theta = df.fem.Constant(self.mesh, df.default_scalar_type(theta))

        # reset the initial conditions
        self.setup_boundaryconditions()
        
        # first solve both Stokes systems
        self.solve_stokes_isoviscous(petsc_options=petsc_options_s)

        # retrieve the temperature forms
        ST, fT, _ = self.temperature_forms()
        solver_T = TemperatureSolver(ST, fT, self.bcs_T, self.T_i, 
                                     petsc_options=petsc_options_T)

        # and solve the temperature problem repeatedly with time step dt
        t = 0
        ti = 0
        tf_nd = tf/self.t0_Myr
        if self.comm.rank == 0 and verbosity>0:
            print("Entering timeloop with {:d} steps (dt = {:g} Myr, final time = {:g} Myr)".format(int(np.ceil(tf_nd/self.dt.value)), dt, tf,))
        # enter the time-loop
        while t < tf_nd - 1e-9:
            if self.comm.rank == 0 and verbosity>1:
                print("Step: {:>6d}, Times: {:>9g} -> {:>9g} Myr".format(ti, t*self.t0_Myr, (t+self.dt.value)*self.t0_Myr))
            if plotter is not None:
                for mesh in plotter.meshes:
                    if self.T_i.name in mesh.point_data:
                        mesh.point_data[self.T_i.name][:] = self.T_i.x.array
                plotter.write_frame()
            # set the old solution to the new solution
            self.T_n.x.array[:] = self.T_i.x.array
            # solve for the new solution
            self.T_i = solver_T.solve()
            # increment the timestep number
            ti+=1
            # increment the time
            t+=self.dt.value
        if self.comm.rank == 0 and verbosity>0:
            print("Finished timeloop after {:d} steps (final time = {:g} Myr)".format(ti, t*self.t0_Myr,))

        # only update the pressure at the end as it is not necessary earlier
        self.update_p_functions()

#### Demonstration - Benchmark case 1 (time-dependent)

In [ ]:
resscale = 5.0
xs = [0.0, 140.0, 240.0, 400.0]
ys = [0.0, -70.0, -120.0, -200.0]
lc_depth = 40
uc_depth = 15
coast_distance = 0
extra_width = 0
sztype = 'continental'
io_depth = 139
A      = 100.0      # age of subducting slab (Myr)
qs     = 0.065      # surface heat flux (W/m^2)
Vs     = 100.0      # slab speed (mm/yr)

slab = create_slab(xs, ys, resscale, lc_depth)
geom_case1td = create_sz_geometry(slab, resscale, sztype, io_depth, extra_width, 
                              coast_distance, lc_depth, uc_depth)
sz_case1td = TDIsoSubductionProblem(geom_case1td, A=A, Vs=Vs, sztype=sztype, qs=qs)

sz_case1td.solve(25, 0.05, theta=0.5)
    

In [ ]:
plotter_isotd = utils.plot.plot_scalar(sz_case1td.T_i, scale=sz_case1td.T0, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'Temperature (deg C)', 'bold':True})
utils.plot.plot_vector_glyphs(sz_case1td.vw_i, plotter=plotter_isotd, factor=0.1, gather=True, color='k', scale=utils.mps_to_mmpyr(sz_case1td.v0))
utils.plot.plot_vector_glyphs(sz_case1td.vs_i, plotter=plotter_isotd, factor=0.1, gather=True, color='k', scale=utils.mps_to_mmpyr(sz_case1td.v0))
sz_case1td.geom.pyvistaplot(plotter=plotter_isotd, color='green', width=2)
cdpt = sz_case1td.geom.slab_spline.findpoint('Slab::FullCouplingDepth')
utils.plot.plot_points([[cdpt.x, cdpt.y, 0.0]], plotter=plotter_isotd, render_points_as_spheres=True, point_size=10.0, color='green')
utils.plot.plot_show(plotter_isotd)
utils.plot.plot_save(plotter_isotd, output_folder / "sz_problem_case1td_solution.png")

## Finish up

Convert this notebook to a python module (saving first and ignoring markdown cells and those tagged as "main" or "ipy").

In [ ]:
from ipylab import JupyterFrontEnd
app = JupyterFrontEnd()
app.commands.execute('docmanager:save')
!jupyter nbconvert --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags="['main', 'ipy']" --TemplateExporter.exclude_markdown=True --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True --NbConvertApp.export_format=script --ClearOutputPreprocessor.enabled=True --FilesWriter.build_directory=../../python/sz_problems --NbConvertApp.output_base=sz_tdep_isoviscous 3.5b_sz_tdep_isoviscous.ipynb